In [1]:
import os
import h5py

In [2]:
import librosa

In [3]:
import itertools
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
import numpy as np
import pandas as pd

In [6]:
from scipy.stats import kurtosis
from scipy.stats import skew

In [7]:
from collections import defaultdict
import pickle

In [8]:
import ast

In [9]:
#I took help from github link: https://github.com/celestinhermez/music-genre-classification/blob/master/data_preprocessing/create_audio_features.ipynb
# path to the small directory
SMALL_AUDIO_DIR = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small'

# function to get the paths to all the songs in the small dataset
def audio_paths(AUDIO_DIR):
    AUDIO_PATHS = []
    # iterate through all the directories with songs in them
    for path in [os.path.join('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small', p) 
                 for p in os.listdir('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small') 
                 if not (p.endswith('checksums') or p.endswith('.txt') or p.endswith('.DS_Store'))]:
        # add all songs to the list
        AUDIO_PATHS = AUDIO_PATHS + [os.path.join(path, track).replace('\\', '/') for track in os.listdir(path)]
    
    return AUDIO_PATHS

# store all the small paths
SMALL_PATHS = audio_paths(SMALL_AUDIO_DIR)

In [10]:
print(SMALL_PATHS[0:15])

['E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000002.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000005.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000010.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000140.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000141.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000148.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000182.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000190.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000193.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000194.mp3', 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000197.mp3', 'E:/8th Semester/CSE4836-Patter

In [11]:
t=pd.read_csv('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/tracks.csv')
t.head(3)

C:\Users\HP\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food


In [12]:
# function to load metadata
# adapted from https://github.com/mdeff/fma/blob/master/utils.py
def metadata_load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    pd.CategoricalDtype(categories=SUBSETS, ordered=True))
        except ValueError:
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks

In [13]:
# function to get genre information for each track ID
def track_genre_information(GENRE_PATH, TRACKS_PATH, FILE_PATHS, subset):
    """
    GENRE_PATH (str): path to the csv with the genre metadata
    TRACKS_PATH (str): path to the csv with the track metadata
    FILE_PATHS (list): list of paths to the mp3 files
    subset (str): the subset of the data desired
    """
    # get the genre information
    genres = pd.read_csv(GENRE_PATH)

    # load metadata on all the tracks
    tracks = metadata_load(TRACKS_PATH)
    print(tracks.head(5))

    # focus on the specific subset tracks
    subset_tracks = tracks[tracks['set', 'subset'] <= subset]

    # extract track ID and genre information for each track
    subset_tracks_genre = np.array([np.array(subset_tracks.index), 
                                  np.array(subset_tracks['track', 'genre_top'])]).T
    
    # extract track indices from the file paths
    track_indices = []
    for path in FILE_PATHS:
        track_indices.append(path.split('/')[-1].split('.')[0].lstrip('0'))

    # get the genre associated with each file path, thanks to the path ID
    print(track_indices)
    track_indices = pd.DataFrame({'file_path':FILE_PATHS,'track_id':np.array(track_indices).astype(int)})
    tracks_genre_df = pd.DataFrame({'track_id': subset_tracks_genre[:,0], 'genre': subset_tracks_genre[:,1]})
    track_genre_data = track_indices.merge(tracks_genre_df, how='left')
    
    # label classes with numbers
    encoder = LabelEncoder()
    track_genre_data['genre_nb'] = encoder.fit_transform(track_genre_data.genre)
    
    return track_genre_data

# get genre information for all tracks from the small subset
GENRE_PATH = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/genres.csv'
TRACKS_PATH = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_metadata/tracks.csv'
subset = 'small'

small_tracks_genre = track_genre_information(GENRE_PATH, TRACKS_PATH, SMALL_PATHS, subset)

            album                                                          \
         comments        date_created date_released engineer favorites id   
track_id                                                                    
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4  6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   

                                                                           \
                                        information listens producer tags   
track_id                                                                    
2                                           <p></p>    6073      NaN   []   
3                                           <p></p>    6073      NaN   []  

In [14]:
small_tracks_genre.head(4)

,file_path,track_id,genre,genre_nb
0,E:/8th Semester/CSE4836-Pattern Recognition La...,2,Hip-Hop,3
1,E:/8th Semester/CSE4836-Pattern Recognition La...,5,Hip-Hop,3
2,E:/8th Semester/CSE4836-Pattern Recognition La...,10,Pop,6
3,E:/8th Semester/CSE4836-Pattern Recognition La...,140,Folk,2


In [15]:
print(small_tracks_genre.shape)

(7997, 4)


In [16]:
small_tracks_genre = small_tracks_genre.drop([120,121,122,123,124,125], axis=0)

In [17]:
print(small_tracks_genre.shape)

(7991, 4)


In [18]:
import scipy.io.wavfile as wav
import numpy as np
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator
import math
import numpy as np
import os
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew

In [19]:
def create_small_sample(X, y, window = 0.25, overlap = 0.5):
  temp_X = []
  temp_y = []

  row = X.shape[0]
  sample_on_window = int(row*window)
  iter = int(sample_on_window*(1.-overlap))
    
  last_iter = row - sample_on_window + iter

  splitted_song = [X[i:i+sample_on_window] for i in range(0, last_iter , iter)]
  for s in splitted_song:
      if s.shape[0] == sample_on_window:
        temp_X.append(s)
        temp_y.append(y)

  return np.array(temp_X), np.array(temp_y)

In [20]:
def to_mel(song, n_fft=1024, hop_length=256):
  return librosa.feature.melspectrogram(song, n_fft=n_fft,hop_length=hop_length, n_mels=128)[:,:,np.newaxis]

In [21]:
def get_melspectrogram(song, n_fft=1024, hop_length=256):
  mels_by_split = map(to_mel, song)
  return np.array(list(mels_by_split))

In [22]:
def split_then_mels(X, y):
  arr_splits = []
  arr_genres = []
  f = 0
  for feat, genre in zip(X, y):
    print("Reading file: {}".format(feat))
    print(f)
    f = f+1
    signal, rate = librosa.load(feat)
    signal = signal[:take_samples]

    # Splitsongs into 0.15 of each to take sample of 1.5 seconds only
    signals, y = create_small_sample(signal, genre)

    # Get Labrosa Melspectogram
    splits = get_melspectrogram(signals)

        
    arr_genres.extend(y)
    arr_splits.extend(splits)
    
  #ret_files = np.array(arr_splits)
  #y_val = np.array(arr_genres)
  return arr_splits, arr_genres

In [23]:
take_samples = 660000
#Creating a Dictionary of Genres
genres = {
    'Hip-Hop': 0, 'Pop': 1, 'Folk': 2, 'Experimental': 3, 'Rock': 4, 
    'International': 5, 'Electronic': 6, 'Instrumental': 7
}

In [24]:
small_tracks_genre = small_tracks_genre.sample(frac=1, random_state=66)

In [25]:
small_tracks_genre=small_tracks_genre.groupby('genre', group_keys=False).apply(lambda x: x.sample(125))

In [26]:
print(small_tracks_genre.value_counts(subset='genre'))

genre
Rock             125
Pop              125
International    125
Instrumental     125
Hip-Hop          125
Folk             125
Experimental     125
Electronic       125
dtype: int64


In [27]:
list_file = []
list_genres = []

for ind in small_tracks_genre.index:
  song_link = small_tracks_genre['file_path'][ind]
  song_genre = small_tracks_genre['genre'][ind]
  list_file.append(song_link)
  list_genres.append(genres[song_genre])

X_train, X_test, y_train, y_test = train_test_split(list_file, list_genres, test_size=0.15, random_state=47, stratify=list_genres)

In [28]:
print(len(X_train) , len(y_train), len(X_test), len(y_test))

850 850 150 150


In [29]:
sample_x_train = len(X_train)
sample_x_test = len(X_test)

In [30]:
X_return, y_return = split_then_mels(X_train, y_train)

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/131/131536.mp3
0


C:\Users\HP\Anaconda\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/132/132794.mp3
1


C:\Users\HP\Anaconda\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/072/072058.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/069826.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/105/105683.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081436.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/112/112481.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/037/037417.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122355.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/109/109497.mp3
9
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/003/003534.mp3
10
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/069/06

...

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/047/047663.mp3
483
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129088.mp3
484
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107025.mp3
485
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/055/055572.mp3
486
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130920.mp3
487
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/078/078847.mp3
488
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004785.mp3
489
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004521.mp3
490
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/106/106569.mp3
491
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130993.mp3
563
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/003/003722.mp3
564
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/142/142569.mp3
565
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/004/004100.mp3
566
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/094/094465.mp3
567
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/098/098578.mp3
568
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064630.mp3
569
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/060/060041.mp3
570
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/081/081802.mp3
571
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/000/000777.mp3
643
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130951.mp3
644
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126489.mp3
645
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/129/129391.mp3
646
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/057/057939.mp3
647
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/089/089196.mp3
648
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/120/120782.mp3
649
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/152/152254.mp3
650
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/119/119938.mp3
651
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/127/127276.mp3
723
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/130/130956.mp3
724
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/087/087430.mp3
725
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/062/062337.mp3
726
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121450.mp3
727
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/039/039607.mp3
728
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107048.mp3
729
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122082.mp3
730
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/040/040542.mp3
731
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/126/126743.mp3
803
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/125/125191.mp3
804
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/101/101873.mp3
805
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/070/070774.mp3
806
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/113/113343.mp3
807
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/054/054151.mp3
808
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/014/014744.mp3
809
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/064/064253.mp3
810
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/115/115922.mp3
811
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset

In [31]:
w = len(X_return)
print(w)

5950


In [32]:
sample_per_music_file = int(w/sample_x_train)
print(sample_per_music_file)

7


In [33]:
temp_arr = np.array(X_return)
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX_new', temp_arr)

In [34]:
temp_arr = np.array(y_return)
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datay_new', temp_arr)

In [35]:
X_return, y_return = split_then_mels(X_test, y_test)

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/026/026174.mp3
0
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/107/107050.mp3
1
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/122/122630.mp3
2
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/085/085422.mp3
3
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121651.mp3
4
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/073/073771.mp3
5
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/038/038560.mp3
6
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/085/085425.mp3
7
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/111/111188.mp3
8
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/056/056

Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/072/072074.mp3
81
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/121/121257.mp3
82
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/024/024975.mp3
83
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/047/047201.mp3
84
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/110/110648.mp3
85
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/080/080776.mp3
86
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/074/074382.mp3
87
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/137/137896.mp3
88
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_small/114/114279.mp3
89
Reading file: E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/fma_smal

In [36]:
w = len(X_return)
print(w)

1050


In [37]:
temp_arr = np.array(X_return)
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datatestX_new', temp_arr)

In [38]:
temp_arr = np.array(y_return)
np.save('E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datatesty_new', temp_arr)

In [39]:
del temp_arr
del X_return
del y_return

# CNN Model

In [1]:
import os
import h5py
import itertools
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import tensorflow as tf
from tensorflow import keras

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
np.random.seed(467)

In [3]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/dataX_new.npy'
X_train = np.load(fileXtrain)

In [4]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datay_new.npy'
y_train = np.load(fileXtrain)

In [5]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datatestX_new.npy'
X_test = np.load(fileXtrain)

In [6]:
fileXtrain = 'E:/8th Semester/CSE4836-Pattern Recognition Lab/FMA_Dataset/Dataset/ForCNNList/datatesty_new.npy'
y_test = np.load(fileXtrain)

In [7]:
print(X_train.shape, X_train.shape, y_train.shape, y_test.shape)

(5950, 128, 645, 1) (5950, 128, 645, 1) (5950,) (1050,)


In [8]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
print(X_train.shape, X_train.shape, y_train.shape, y_test.shape)

(5950, 128, 645, 1) (5950, 128, 645, 1) (5950, 8) (1050, 8)


In [10]:
genres = {
    'Hip-Hop': 0, 'Pop': 1, 'Folk': 2, 'Experimental': 3, 'Rock': 4, 
    'International': 5, 'Electronic': 6, 'Instrumental': 7
}

# Data Augmentation

In [11]:
from tensorflow.keras.utils import Sequence
class GTZANGenerator(Sequence):
    def __init__(self, X, y, batch_size=64, is_test = False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.is_test = is_test
    
    def __len__(self):
        return int(np.ceil(len(self.X)/self.batch_size))
    
    def __getitem__(self, index):
        # Get batch indexes
        signals = self.X[index*self.batch_size:(index+1)*self.batch_size]

        # Apply data augmentation
        if not self.is_test:
            signals = self.__augment(signals)
        return signals, self.y[index*self.batch_size:(index+1)*self.batch_size]
    
    def __augment(self, signals, hor_flip = 0.5, random_cutout = 0.5):
        spectrograms =  []
        for s in signals:
            signal = copy(s)
            
            # Perform horizontal flip
            if np.random.rand() < hor_flip:
                signal = np.flip(signal, 1)

            # Perform random cutoout of some frequency/time
            if np.random.rand() < random_cutout:
                lines = np.random.randint(signal.shape[0], size=3)
                cols = np.random.randint(signal.shape[0], size=4)
                signal[lines, :, :] = -80 # dB
                signal[:, cols, :] = -80 # dB

            spectrograms.append(signal)
        return np.array(spectrograms)
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
        np.random.shuffle(self.indexes)
        return None

# Defining The Network

In [12]:
def conv_block(x, n_filters, pool_size=(2, 2)):
    x = Conv2D(n_filters, (3, 3), strides=(1, 1), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=pool_size, strides=pool_size)(x)
    x = Dropout(0.25)(x)
    return x

In [13]:
# Model Definition
def create_model(input_shape, num_genres):
    inpt = Input(shape=input_shape)
    x = conv_block(inpt, 16)
    x = conv_block(x, 32)
    x = conv_block(x, 64)
    x = conv_block(x, 128)
    x = conv_block(x, 256)
    
    # Global Pooling and MLP
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', 
              kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    x = Dropout(0.25)(x)
    predictions = Dense(num_genres, 
                        activation='softmax', 
                        kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    
    model = Model(inputs=inpt, outputs=predictions)
    return model

In [14]:
model = create_model(X_train[0].shape, 8)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 645, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 645, 16)      160       
_________________________________________________________________
activation (Activation)      (None, 128, 645, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 322, 16)       0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 322, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 322, 32)       4640      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 322, 32)       0     

In [16]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [17]:
reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.95,
    patience=3,
    verbose=1,
    mode='min',
    min_delta=0.0001,
    cooldown=2,
    min_lr=1e-5
)

In [18]:
# Generators
batch_size = 128
train_generator = GTZANGenerator(X_train, y_train)
steps_per_epoch = np.ceil(len(X_train)/batch_size)

validation_generator = GTZANGenerator(X_test, y_test)
val_steps = np.ceil(len(X_test)/batch_size)

In [19]:
import tensorflow as tf
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available 1


In [20]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)

config.gpu_options.per_process_gpu_memory_fraction = 0.3
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

Num GPUs Available:  1


In [21]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=val_steps,
    epochs=250,
    verbose=1,
    callbacks=[reduceLROnPlat])

C:\Users\HP\Anaconda\envs\tf-fma\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/250
47/47 [==============================] - 163s 2s/step - loss: 53.9054 - accuracy: 0.1276 - val_loss: 18.7514 - val_accuracy: 0.1875
Epoch 2/250
47/47 [==============================] - 113s 2s/step - loss: 17.2694 - accuracy: 0.1736 - val_loss: 13.4941 - val_accuracy: 0.2170
Epoch 3/250
47/47 [==============================] - 96s 2s/step - loss: 12.6268 - accuracy: 0.1909 - val_loss: 10.3384 - val_accuracy: 0.1528
Epoch 4/250
47/47 [==============================] - 96s 2s/step - loss: 9.7896 - accuracy: 0.1467 - val_loss: 8.2853 - val_accuracy: 0.2118
Epoch 5/250
47/47 [==============================] - 96s 2s/step - loss: 7.8462 - accuracy: 0.1908 - val_loss: 6.8698 - val_accuracy: 0.1788
Epoch 6/250
47/47 [==============================] - 96s 2s/step - loss: 6.5500 - accuracy: 0.2147 - val_loss: 5.7538 - val_accuracy: 0.2465
Epoch 7/250
47/47 [==============================] - 96s 2s/step - loss: 5.5436 - accuracy: 0.2324 - val_loss: 5.0833 - val_accuracy: 0.1615
Epoch

Epoch 57/250
47/47 [==============================] - 96s 2s/step - loss: 1.8328 - accuracy: 0.3538 - val_loss: 1.8160 - val_accuracy: 0.3611
Epoch 58/250
47/47 [==============================] - 96s 2s/step - loss: 1.9489 - accuracy: 0.3344 - val_loss: 1.7885 - val_accuracy: 0.3524
Epoch 59/250
47/47 [==============================] - 96s 2s/step - loss: 1.7432 - accuracy: 0.3825 - val_loss: 1.7736 - val_accuracy: 0.3733

Epoch 00059: ReduceLROnPlateau reducing learning rate to 0.0008145062311086804.
Epoch 60/250
47/47 [==============================] - 96s 2s/step - loss: 1.8574 - accuracy: 0.3532 - val_loss: 1.8777 - val_accuracy: 0.3924
Epoch 61/250
47/47 [==============================] - 96s 2s/step - loss: 1.7892 - accuracy: 0.3646 - val_loss: 1.7412 - val_accuracy: 0.3906
Epoch 62/250
47/47 [==============================] - 96s 2s/step - loss: 1.8140 - accuracy: 0.3431 - val_loss: 1.7699 - val_accuracy: 0.4115
Epoch 63/250
47/47 [==============================] - 96s 2s/step -

47/47 [==============================] - 96s 2s/step - loss: 1.5773 - accuracy: 0.4705 - val_loss: 1.6583 - val_accuracy: 0.4201
Epoch 109/250
47/47 [==============================] - 96s 2s/step - loss: 1.6111 - accuracy: 0.4853 - val_loss: 1.7245 - val_accuracy: 0.3941

Epoch 00109: ReduceLROnPlateau reducing learning rate to 0.00046329112810781223.
Epoch 110/250
47/47 [==============================] - 96s 2s/step - loss: 1.6020 - accuracy: 0.4770 - val_loss: 1.6906 - val_accuracy: 0.4479
Epoch 111/250
47/47 [==============================] - 96s 2s/step - loss: 1.5646 - accuracy: 0.5035 - val_loss: 1.7496 - val_accuracy: 0.4410
Epoch 112/250
47/47 [==============================] - 96s 2s/step - loss: 1.5832 - accuracy: 0.4633 - val_loss: 1.7059 - val_accuracy: 0.4253
Epoch 113/250
47/47 [==============================] - 96s 2s/step - loss: 1.6067 - accuracy: 0.4731 - val_loss: 1.7192 - val_accuracy: 0.4288

Epoch 00113: ReduceLROnPlateau reducing learning rate to 0.00044012657308

47/47 [==============================] - 96s 2s/step - loss: 1.3708 - accuracy: 0.5735 - val_loss: 1.7877 - val_accuracy: 0.4306

Epoch 00159: ReduceLROnPlateau reducing learning rate to 0.0002635200624354184.
Epoch 160/250
47/47 [==============================] - 96s 2s/step - loss: 1.3797 - accuracy: 0.5441 - val_loss: 1.6937 - val_accuracy: 0.4774
Epoch 161/250
47/47 [==============================] - 96s 2s/step - loss: 1.3897 - accuracy: 0.5196 - val_loss: 1.6945 - val_accuracy: 0.4514
Epoch 162/250
47/47 [==============================] - 96s 2s/step - loss: 1.3295 - accuracy: 0.5874 - val_loss: 1.6437 - val_accuracy: 0.4497
Epoch 163/250
47/47 [==============================] - 96s 2s/step - loss: 1.2658 - accuracy: 0.5961 - val_loss: 1.8224 - val_accuracy: 0.4340

Epoch 00163: ReduceLROnPlateau reducing learning rate to 0.0002503440482541919.
Epoch 164/250
47/47 [==============================] - 96s 2s/step - loss: 1.3301 - accuracy: 0.5618 - val_loss: 1.7371 - val_accuracy: 0

Epoch 209/250
47/47 [==============================] - 96s 2s/step - loss: 1.1390 - accuracy: 0.6628 - val_loss: 1.8673 - val_accuracy: 0.4566
Epoch 210/250
47/47 [==============================] - 96s 2s/step - loss: 1.1622 - accuracy: 0.6447 - val_loss: 1.8383 - val_accuracy: 0.4653
Epoch 211/250
47/47 [==============================] - 96s 2s/step - loss: 1.1205 - accuracy: 0.6633 - val_loss: 1.8370 - val_accuracy: 0.4410

Epoch 00211: ReduceLROnPlateau reducing learning rate to 0.00013527592891477978.
Epoch 212/250
47/47 [==============================] - 96s 2s/step - loss: 1.1870 - accuracy: 0.6173 - val_loss: 1.8199 - val_accuracy: 0.4375
Epoch 213/250
47/47 [==============================] - 96s 2s/step - loss: 1.1128 - accuracy: 0.6481 - val_loss: 1.7805 - val_accuracy: 0.4531
Epoch 214/250
47/47 [==============================] - 96s 2s/step - loss: 1.2243 - accuracy: 0.6070 - val_loss: 1.8018 - val_accuracy: 0.4583
Epoch 215/250
47/47 [==============================] - 96s 2

In [22]:
model.save("cnn_final3.h5")

In [23]:
score = model.evaluate(X_test, y_test, verbose=0)
print("val_loss = {:.3f} and val_acc = {:.3f}".format(score[0], score[1]))

val_loss = 2.315 and val_acc = 0.413


In [24]:
new_model = tf.keras.models.load_model('cnn_final3.h5')
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 645, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 645, 16)      160       
_________________________________________________________________
activation (Activation)      (None, 128, 645, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 322, 16)       0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 322, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 322, 32)       4640      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 322, 32)       0     

# Max-Voting Approach

In [25]:
sample_per_music_file = 7

In [26]:
y_pred = new_model.predict(X_test)

In [27]:
print(y_pred.shape[0])
print(y_pred.shape[1])

1050
8


In [28]:
def evaluate_performance(y_pred,y_test):
    total_files = int(y_test.shape[0] / sample_per_music_file)
    no_of_right_pred = 0
    no_of_wrong_pred = 0
    for i in range(0, total_files):
        start_at = i * sample_per_music_file 
        end_at = start_at + sample_per_music_file
        count_in = np.zeros(y_pred.shape[1])
        for j in range(start_at, end_at):
            label_now = np.argmax(y_pred[j])
            count_in[label_now] = count_in[label_now] + 1
        pred_label = np.argmax(count_in)
        true_label = np.argmax(y_test[start_at])
        
        if pred_label == true_label:
            no_of_right_pred = no_of_right_pred + 1
        else:
            no_of_wrong_pred = no_of_wrong_pred + 1
            
    print('Right:',no_of_right_pred)
    print('Wrong:',no_of_wrong_pred)
    
    accuracy = no_of_right_pred / total_files
    
    return accuracy
            
            
        

In [29]:
y_pred = new_model.predict(X_test)
calculate_accuracy = evaluate_performance(y_pred,y_test)
print("Test_acc = {:.3f}".format(calculate_accuracy))

Right: 66
Wrong: 84
Test_acc = 0.440
